In [ ]:
! pip install cohere langchain qdrant-client tfds-nightly


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.3/206.3 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
from langchain.document_loaders import PyPDFLoader
import textwrap as tr
import random
import os

api_key = 'zpeSL6UKGjJPNBOdoro7IYIWMhIzDU3OCLXQSlBG'

In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 4.6 MB/s eta 0:00:00


In [ ]:
#from google.colab import files
#upload = files.upload()

In [ ]:
# save the embeddings in a DB that is persistent
loader = PyPDFLoader("/content/drive/My Drive/Neurals.pdf")
document=loader.load_and_split()
print(len(document))
docs=""
for i in range(len(document)):
  docs=docs+" "+document[i].page_content
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0,length_function = len, )
docs = text_splitter.create_documents([docs])
texts = [doc.page_content for doc in docs]

# Define the embeddings model
embeddings = CohereEmbeddings(model = 'multilingual-22-12', cohere_api_key=api_key)
# Embed the documents and store in index
db = Qdrant.from_texts(texts, embeddings, location=":memory:", collection_name="my_documents", distance_func="Dot")
print(texts)

494
['Neural Networks for \nPattern Recognition \nCHRISTOPHER M. BISHOP \nDepartment of Computer Science \nand Applied Mathematics \nAston University \nBirmingham, UK \nCLARENDON PRESS • OXFORD \n1995 FOREWORD \nGeoffrey Hinton \nDepartment of Computer Science \nUniversity of Toronto \nFor those entering the field of artificial neural networks, there has been an acute \nneed for an authoritative textbook that explains the main ideas clearly and con\xad\nsistently using the basic tools of linear algebra, calculus, and simple probability \ntheory. There have been many attempts to provide such a text, but until now, \nnone has succeeded. Some authors have failed to separate the basic ideas and \nprinciples from the soft and fuzzy intuitions that led to some of the models as \nwell as to most of the exaggerated claims. Others have been unwilling to use the \nbasic mathematical tools that are essential for a rigorous understanding of the', 'material. Yet others have tried to cover too many 

In [ ]:
queries = ["How is error function defined?",
           "What is neural network?",
           "What is Bayes Theorem?"]

In [ ]:
answers = []
for query in queries:
  docs= db.similarity_search(query)
  print(len(docs))
  for i in range(len(docs)):
    answers.append(docs[i].page_content)

4
4
4


In [ ]:
answers

["In general the error function is given by a sum of terms each of which is \ncalculated using just one of the patterns from the training set, so that \n£(w) = ]T)£n(w) (3.53) \nn (3.52) 96 3: Single-Layer Networks \nwhere the term En is calculated using pattern n only. In this case we can update \nthe weight vector using just one pattern at a time \n(T+I) (T) 9En \nand this is repeated many times by cycling through all of the patterns used in the \n.definition of E. This form of sequential, or pattern-based, update is reminiscent \nof the Robbins-Monro procedure introduced in Section 2.4, and many of the \nsame comments apply here. In particular, this technique allows the system to be \nused in real-time adaptive applications in which data is arriving continuously. \nEach data point can be used once and then discarded, and if the value of r\\ \nis chosen appropriately, the system may be able to 'track' any slow changes \nin the characteristics of the data. If r) is chosen to decrease 

In [ ]:
retriever=db.as_retriever()

from langchain.agents.agent_toolkits import create_retriever_tool
tool = create_retriever_tool(
    retriever,
    "Neurals",
    "Searches and returns documents regarding the Neurals."
)
tools = [tool]


In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatCohere
from langchain.agents.agent_types import AgentType
llm = ChatCohere(temperature = 0)
memory = ConversationBufferMemory(memory_key="chat_history")

from langchain.agents import initialize_agent, create_react_agent

conversational_agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
    handle_parsing_errors=True,
)


In [ ]:
result = conversational_agent("What is Neural Network")
print(result)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Neurals
Action Input: Neural Network
Observation: Neural Network is a framework used to process and interpret information. It is a method of computing where the information traverses from the input layer to the output layer through hidden layers which are interconnected nodes. These nodes are inspired by the human brain and the way it interprets and learns information through signal transmission. 

Would you like to know more about Neural Networks or any other topic? 

Thought: Do I need to use a tool? No
AI: It is fascinating how the Neural Network model imitates the human brain and its learning process. Though, Neural Networks have their limitations and mathematical flaws. Would you like me to give you some points on that front as well?

> Finished chain.
{'input': 'What is Neural Network', 'chat_history': '', 'output': 'It is fascinating how the Neural Network model imitates the human brain and its

In [ ]:
result = conversational_agent("What is Bayes Theorem?")
print(result)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Bayes' theorem is primarily used to calculate the probability of a specific event occurring, which is denoted as P(A), given the occurrence of another event, denoted as P(B). It allows us to assess the likelihood of one event in the context of another event's occurrence.

This theorem is of significant importance to mathematicians, statisticians, and computer scientists as it helps solve problems and make predictions through Bayesian inference, a method used for drawing conclusions about the parameters of a population based on information pertaining to a sample.

Would you like me to apply Bayes' theorem to a specific scenario? I can do that for you!

> Finished chain.
{'input': 'What is Bayes Theorem?', 'chat_history': 'Human: What is Neural Network\nAI: It is fascinating how the Neural Network model imitates the human brain and its learning process. Though, Neural Networks have their limitations and math